In [1]:
import json
import pandas as pd

### Process person data

#### Auxiliary functions

In [2]:
# We need this function to replace all occurences of {'-self-closing': 'true'} by None.
def remove_self_closing(elem):
    if type(elem) != dict and type(elem) != list:
        return
    
    if type(elem) == list:
        for idx, value in enumerate(elem):
            if value == {'-self-closing': 'true'}:
                elem[key] = None
            elif type(value) == dict or type(value) == list:
                remove_self_closing(value)
    
    if type(elem) == dict:
        for (key, value) in elem.items():
            if value == {'-self-closing': 'true'}:
                elem[key] = None
            elif type(value) == dict  or type(value) == list:
                remove_self_closing(value)

#### Import converted/MDB_STAMMDATEN.json, reorganize the data and remove parts that are not interesting for us

In [3]:
with open("__converted/MDB_STAMMDATEN.json", "r") as f:
    data = json.load(f);

persons_raw = data["DOCUMENT"]["MDB"]

persons = []
for p in persons_raw:
    person = {}
    
    name = p["NAMEN"]["NAME"]
    name = name[-1] if type(name) == list else name
    bio = p["BIOGRAFISCHE_ANGABEN"]
    wp_raw = p["WAHLPERIODEN"]["WAHLPERIODE"]
    wp_raw = [wp_raw] if type(wp_raw) != list else wp_raw
    
    # We don't need the data if the person is/was not a MP in the current term
    if "19" not in [wp["WP"] for wp in wp_raw]:
        continue
    
    person["nachname"]      = name["NACHNAME"]
    person["vorname"]       = name["VORNAME"]
    person["geburtsdatum"]  = bio["GEBURTSDATUM"]
    person["geburtsort"]    = bio["GEBURTSORT"]
    person["sterbedatum"]   = bio["STERBEDATUM"]
    person["partei"]        = bio["PARTEI_KURZ"]
    person["geschlecht"]    = bio["GESCHLECHT"]
    person["familienstand"] = bio["FAMILIENSTAND"]
    person["religion"]      = bio["RELIGION"]
    person["beruf"]         = bio["BERUF"]
    person["anrede_titel"]  = name["ANREDE_TITEL"]
    person["akad_titel"]    = name["AKAD_TITEL"]
    person["vita"]          = bio["VITA_KURZ"]
    person["partei_id"]     = bio["PARTEI_KURZ"]
    if bio["PARTEI_KURZ"] == "CDU" or bio["PARTEI_KURZ"] == "CSU":
        person["partei_id"] = "CDU/CSU"
        
    wahlperioden = {}
    for wp in wp_raw:
        wahlperiode = {}
        wahlperiode["md_von"]     = wp["MDBWP_VON"]
        wahlperiode["md_bis"]     = wp["MDBWP_BIS"]
        wahlperiode["liste"]      = wp["LISTE"]
        wahlperiode["mandatsart"] = wp["MANDATSART"]
        wahlperiode["wkr_land"]   = wp["WKR_LAND"]
        wahlperiode["wkr_name"]   = wp["WKR_NAME"]
        wahlperiode["wkr_nummer"] = wp["WKR_NUMMER"]
        
        institutionen = wp["INSTITUTIONEN"]["INSTITUTION"]
        institutionen = [institutionen] if type(institutionen) != list else institutionen
        wahlperiode["institutionen"] = institutionen
        
        wahlperioden[wp["WP"]] = wahlperiode
        
    person["wahlperioden"] = wahlperioden
            
    for (key, value) in person.items():
        if value == {'-self-closing': 'true'}:
            person[key] = None

    remove_self_closing(persons)
    
    persons.append(person)

#### Export data to final/stammdaten.json

In [4]:
with open("___final/stammdaten.json", "w") as outfile:
    json.dump(persons, outfile, indent = 4, ensure_ascii = False)

#### Other stuff

In [5]:
# Make sure there are no duplicate names (because we want to use them as id)
for i in range(len(persons)):
    for j in range(i+1, len(persons)):
        p1 = persons[i]
        p2 = persons[j]
        if p1["nachname"] == p2["nachname"] and p1["vorname"] == p2["vorname"]:
            print(p1["nachname"], p1["vorname"])

### Process vote data

In [6]:
CORRECTIONS = {
    ("Özoguz", "Aydan"): ("Özoğuz", "Aydan"),
    ("Dagdelen", "Sevim"): ("Dağdelen", "Sevim")
}

# Remove some columns we don't use, check if all names match a name in stammdaten.json and
# correct them if necessary. Store result in <vote_nr>.json and <vote_nr>.csv
def process_vote_data(filename, vote_nr):
    # Read csv
    csv_data = pd.read_csv("__converted/votes/" + filename, sep=";")
    csv_data = csv_data[["Fraktion/Gruppe", "Name", "Vorname", "ja", "nein", "Enthaltung", "ungültig", "nichtabgegeben"]]

    # Apply corrections for names that differ from stammdaten.json
    names = [(p["nachname"], p["vorname"]) for p in persons]

    for idx, row in csv_data[["Name", "Vorname"]].iterrows():
        name = (row["Name"].split(" (")[0], row["Vorname"])
        if name in CORRECTIONS:
            csv_data.at[idx, "Name"]    = CORRECTIONS[name][0]
            csv_data.at[idx, "Vorname"] = CORRECTIONS[name][1]

    # Check if there are more names that require manual intervention
    for _, row in csv_data[["Name", "Vorname"]].iterrows():
        assert((row["Name"].split(" (")[0], row["Vorname"]) in names)

    # Save as json and csv
    json_path = "___final/votes/" + vote_nr + ".json"
    csv_path  = "___final/votes/" + vote_nr + ".csv"

    csv_data.to_csv(csv_path, sep = ";", index = False)
    csv_data.to_json(json_path, orient = "records", force_ascii = False, indent = 4)

In [7]:
files = [
    ("20191220_2_xls-data.csv", "20191220_2") 
]

for file, vote_nr in files:
    process_vote_data(file, vote_nr)